# Importamos las librerías necesarias para poder realizar el ETL

In [9]:
import pandas as pd
import numpy as np
import os
import sys

ruta_etl = os.path.abspath(os.path.join('..', 'ETL_functs'))
    
from desanida_ import desanida_column, extraer_directores

pd.set_option('display.max_columns', None)

Realizamos la carga de los datasets `movies_dataset.csv` y `credits.csv`

In [2]:
movies_df = pd.read_csv('Datasets/movies_dataset.csv')
print(movies_df.shape)
print(movies_df.dtypes)
movies_df.head(1)

(45466, 24)
adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object


h:\Sistema-de-Recomendacion-de-Peliculas\venv\lib\site-packages\IPython\core\interactiveshell.py:3258: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [3]:
credits_df = pd.read_csv('Datasets/credits.csv')
print(credits_df.dtypes)
print(credits_df.shape)
credits_df['cast'].head(1)

cast    object
crew    object
id       int64
dtype: object
(45476, 3)


0    [{'cast_id': 14, 'character': 'Woody (voice)',...
Name: cast, dtype: object

Debemos de hacer una transformación al tipo de datos de la columna `id` en movies_df debido a que esta la toma como si fuese de tipo str (object), de lo contrario no podremos hacer un merge, además de ello observamos que la columna `popularity` de movies_df es de tipo object, a lo cuál deberemos de transformar a tipo `numeric`

In [4]:
credits_df['id'] = credits_df['id'].astype(str)

In [5]:
movies_df['popularity'] = pd.to_numeric(movies_df['popularity'], errors='coerce').fillna(0.0)

In [6]:
# Unimos ambos datasets
merged_df = pd.merge(movies_df, credits_df, on = 'id', how= 'inner')
print(merged_df.shape)

(45538, 26)


Desanidamos las columnas de `belongs_to_collection`, `genres`, `production_companies`, `production_countries`, `spoken_languages`, en los casos de `cast` y `crew` se decide extraer los 5 actores más representativos de cada película y los directores de las películas utilizando las funciones desanida_column y extraer_directores

In [7]:
merged_df['belongs_to_collection'] = merged_df['belongs_to_collection'].apply(lambda x: desanida_column('name', x))

In [8]:
merged_df['belongs_to_collection']

0                  Toy Story Collection
1                                   NaN
2             Grumpy Old Men Collection
3                                   NaN
4        Father of the Bride Collection
                      ...              
45533                               NaN
45534                               NaN
45535                               NaN
45536                               NaN
45537                               NaN
Name: belongs_to_collection, Length: 45538, dtype: object

In [9]:
merged_df['genres'] = merged_df['genres'].apply(lambda x: desanida_column('name', x))

In [10]:
merged_df['genres']

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45533                 [Drama, Family]
45534                         [Drama]
45535       [Action, Drama, Thriller]
45536                              []
45537                              []
Name: genres, Length: 45538, dtype: object

In [11]:
merged_df['production_companies'] = merged_df['production_companies'].apply(lambda x: desanida_column('name', x))

In [12]:
merged_df['production_companies']

0                                [Pixar Animation Studios]
1        [TriStar Pictures, Teitler Film, Interscope Co...
2                           [Warner Bros., Lancaster Gate]
3                 [Twentieth Century Fox Film Corporation]
4             [Sandollar Productions, Touchstone Pictures]
                               ...                        
45533                                                   []
45534                                        [Sine Olivia]
45535                            [American World Pictures]
45536                                          [Yermoliev]
45537                                                   []
Name: production_companies, Length: 45538, dtype: object

In [13]:
merged_df['production_countries'] = merged_df['production_countries'].apply(lambda x: desanida_column('name', x))

In [14]:
merged_df['production_countries']

0        [United States of America]
1        [United States of America]
2        [United States of America]
3        [United States of America]
4        [United States of America]
                    ...            
45533                        [Iran]
45534                 [Philippines]
45535    [United States of America]
45536                      [Russia]
45537              [United Kingdom]
Name: production_countries, Length: 45538, dtype: object

In [15]:
merged_df['spoken_languages'] = merged_df['spoken_languages'].apply(lambda x: desanida_column('name', x))

In [16]:
merged_df['spoken_languages']

0                  [English]
1        [English, Français]
2                  [English]
3                  [English]
4                  [English]
                ...         
45533                [فارسی]
45534                     []
45535              [English]
45536                     []
45537              [English]
Name: spoken_languages, Length: 45538, dtype: object

* En este punto se decide tomar solo las películas que tengan como mínimo el idioma `English` en la columna `spoken_languages` debido a que las demás películas generan ruido y además de ello se piensa que las personas no frecuentan películas en otros idiomas

In [17]:
merged_df = merged_df[merged_df['spoken_languages'].apply(lambda x: 'English' in x if isinstance(x, list) else False)]

In [18]:
merged_df['cast'] = merged_df['cast'].apply(lambda x: desanida_column('name', x, 5))

In [19]:
merged_df['cast']

0        [Tom Hanks, Tim Allen, Don Rickles, Jim Varney...
1        [Robin Williams, Jonathan Hyde, Kirsten Dunst,...
2        [Walter Matthau, Jack Lemmon, Ann-Margret, Sop...
3        [Whitney Houston, Angela Bassett, Loretta Devi...
4        [Steve Martin, Diane Keaton, Martin Short, Kim...
                               ...                        
45530    [Monty Bane, Lucy Butler, David Grammer, Bill ...
45531    [Lisa Boyle, Kena Land, Zaneta Polard, Don Yan...
45532    [Patrick Bergin, Uma Thurman, David Morrissey,...
45535    [Erika Eleniak, Adam Baldwin, Julie du Page, J...
45537                                                   []
Name: cast, Length: 28787, dtype: object

Extraigo el o los directores de cada pelicula

In [20]:
merged_df['crew'] = merged_df['crew'].apply(extraer_directores)

In [21]:
merged_df['crew']

0          [John Lasseter]
1           [Joe Johnston]
2          [Howard Deutch]
3        [Forest Whitaker]
4          [Charles Shyer]
               ...        
45530           [Ben Rock]
45531      [Aaron Osborne]
45532         [John Irvin]
45535     [Mark L. Lester]
45537      [Daisy Asquith]
Name: crew, Length: 28787, dtype: object

Los valores nulos de los campos `revenue`, `budget` se rellenan por el número 0.

In [22]:
replaced_nulls = ['revenue', 'budget']
merged_df[replaced_nulls] = merged_df[replaced_nulls].fillna(0)
merged_df[replaced_nulls] = merged_df[replaced_nulls].replace(False, float)
merged_df['revenue'] = pd.to_numeric(merged_df['revenue'])
merged_df['budget'] = pd.to_numeric(merged_df['budget'])

Los valores nulos del campo `release_date` se eliminan

In [23]:
merged_df.dropna(subset=['release_date'], inplace= True)

Se verifica que las fechas tengan el formato AAAA-mm-dd, además se crea la columna `release_year` donde extrae el año de la fecha de estreno.

In [24]:
merged_df['release_date'] = pd.to_datetime(merged_df['release_date'], format='%Y-%m-%d', errors='coerce')
fechas_invalidas = merged_df[merged_df['release_date'].isna()]
fechas_validas = merged_df[merged_df['release_date'].notna()]

# Imprime los resultados
print("Filas con fechas válidas en formato AAAA-MM-DD:")
print(fechas_validas)
print("\nFilas con fechas inválidas o en otro formato:")
print(fechas_invalidas)

# Se crea la columna 'release_year' para extraer el año de lanzamiento
merged_df['release_year'] = merged_df['release_date'].dt.year

Filas con fechas válidas en formato AAAA-MM-DD:
       adult           belongs_to_collection    budget  \
0      False            Toy Story Collection  30000000   
1      False                             NaN  65000000   
2      False       Grumpy Old Men Collection         0   
3      False                             NaN  16000000   
4      False  Father of the Bride Collection         0   
...      ...                             ...       ...   
45530  False                             NaN         0   
45531  False                             NaN         0   
45532  False                             NaN         0   
45535  False                             NaN         0   
45537  False                             NaN         0   

                             genres                              homepage  \
0       [Animation, Comedy, Family]  http://toystory.disney.com/toy-story   
1      [Adventure, Fantasy, Family]                                   NaN   
2                 [Roman

In [25]:
merged_df['release_year'].sort_values()

44814    1903
44816    1906
44806    1909
32399    1911
44822    1911
         ... 
33442    2018
44610    2018
38963    2018
30482    2018
26644    2020
Name: release_year, Length: 28771, dtype: int64

Se decide eliminar las películas que son anteriores a la década de los 60's debido a que se cree que no son películas que sean muy frecuentadas por las personas

In [26]:
merged_df = merged_df[merged_df['release_year'] >= 1960]
merged_df.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,release_year
0,False,Toy Story Collection,30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,[English],Released,NaN,Toy Story,False,7.7,5415.0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",[John Lasseter],1995


Se crea la columna con el retorno de inversión, llamada return con los campos `revenue` y `budget`, dividiendo estas dos últimas `revenue` / `budget`, y, cuando no hay datos disponibles para calcularlo, se toma el valor 0.

In [27]:
merged_df['return'] = np.where(merged_df['budget'] != 0, merged_df['revenue'] / merged_df['budget'], 0)

In [28]:
merged_df['return'].value_counts()

0.000000    19921
2.000000        8
4.000000        7
1.000000        6
4.275705        4
            ...  
0.010018        1
4.910743        1
0.122080        1
0.071040        1
1.869934        1
Name: return, Length: 4650, dtype: int64

Se eliminan las columnas que no serán utilizadas, `video`, `imdb_id`, `adult`, `original_title`, `poster_path` y `homepage`.

In [29]:
merged_df.drop(['video', 'adult', 'original_title', 'poster_path', 'homepage'], axis=1, inplace=True)


## EDA (Análisis Exploratorio de los Datos)

Guardamos nuestro DataFrame en una nueva variable

In [30]:
new_df = merged_df

In [31]:
new_df.head(1)

,belongs_to_collection,budget,genres,id,imdb_id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,release_year,return
0,Toy Story Collection,30000000,"[Animation, Comedy, Family]",862,tt0114709,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,[English],Released,NaN,Toy Story,7.7,5415.0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",[John Lasseter],1995,12.451801


Se eligen las columnas `genres` `overview` `cast` `crew` para realizar nuestro modelo, además de las columnas `vote_count` y `popularity` a fin de que podamos ordenar por dichas columnas los resultados

* Generamos un archivo .csv con el fin de guardar nuestros datos transformados y utilizarlos en nuestro modelo de entrenamiento

In [32]:
new_df.to_csv('Datasets/test.csv', index= False)

Importamos las librerías que vamos a utilizar

In [33]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

In [34]:
# Cargamos el archivo CSV
movies_df = pd.read_csv('Datasets/test.csv')

In [35]:
# Normalizamos las columnas'vote_count' y 'popularity' para ponderar
scaler = MinMaxScaler()
movies_df[['vote_count', 'popularity']] = scaler.fit_transform(movies_df[['vote_count', 'popularity']])

In [36]:
# Creamos una columna combinada para 'genres', 'overview', 'cast', y 'crew' para similitud de contenido
movies_df['content'] = (
    movies_df['genres'].fillna('') + " " +
    movies_df['overview'].fillna('') + " " +
    movies_df['cast'].fillna('') + " " +
    movies_df['crew'].fillna('')
)

In [37]:
# Vectorizamos el texto combinado con TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_df['content'])

In [38]:
# Vectorizamos únicamente 'genres' para calcular la similitud específica de géneros
tfidf_genres = TfidfVectorizer(stop_words='english')
genres_matrix = tfidf_genres.fit_transform(movies_df['genres'].fillna(''))

In [39]:
# Calculamos la similitud de coseno en ambas matrices (contenido y géneros)
content_similarity = cosine_similarity(tfidf_matrix)
genre_similarity = cosine_similarity(genres_matrix)

In [40]:
# Construimos la matriz de similitud ponderada sumando los pesos de 'vote_average', 'popularity' y 'genres'
weighted_similarity = (
    content_similarity +
    (movies_df['vote_average'].values[:, None] * 0.25) +
    (movies_df['popularity'].values[:, None] * 0.5) +
    (genre_similarity * 0.25)  # Ponderación de géneros
)

In [41]:
# Cambiamos el tipo de dato para hacer menos pesado la matriz de similaridad
weighted_similarity = weighted_similarity.astype(np.float16)

In [42]:
# Realizamos filtrado colaborativo (SVD)
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(movies_df[['id', 'vote_average', 'vote_count']], reader)
trainset, testset = train_test_split(data, test_size=0.2)
svd = SVD()
svd.fit(trainset)

In [43]:
# Definimos nuestra función de recomendación avanzada
def recomendacion(title, n_recommendations=5):
    # Verificar si el título existe en la base de datos
    if title not in movies_df['title'].values:
        print("El título no se encuentra en la base de datos.")
        return
    
    # Encontrar el índice de la película de interés
    movie_index = movies_df[movies_df['title'] == title].index[0]
    
    # Obtener la colección de la película (si existe)
    collection_name = movies_df.loc[movie_index, 'belongs_to_collection']
    
    # Obtener índices de películas similares
    similarity_scores = list(enumerate(weighted_similarity[movie_index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similar_movies_indices = [i[0] for i in similarity_scores[1:n_recommendations + 10]]  # Obtener suficientes películas
    
    # Filtrar y priorizar recomendaciones
    recommendations = movies_df.iloc[similar_movies_indices]
    if pd.notna(collection_name):  # Si la película pertenece a una colección
        # Películas de la misma colección primero
        collection_movies = recommendations[recommendations['belongs_to_collection'] == collection_name]
        # Luego las películas similares por géneros y ordenadas por popularidad y votos
        other_movies = recommendations[recommendations['belongs_to_collection'] != collection_name]
        other_movies = other_movies.sort_values(by=['popularity', 'vote_count'], ascending=False)
        
        # Combinar ambas listas
        final_recommendations = pd.concat([collection_movies, other_movies]).head(n_recommendations)
    else:
        # Si no pertenece a una colección, solo ordenar por popularidad y votos
        final_recommendations = recommendations.sort_values(by=['popularity', 'vote_count'], ascending=False).head(n_recommendations)
    
    print(f"Películas recomendadas para '{title}':")
    for _, row in final_recommendations.iterrows():
        print(f"{row['title']} - Géneros: {row['genres']}, Puntuación: {row['vote_average']}, Popularidad: {row['popularity']}, Reparto: {row['cast']}, Director: {row['crew']}")



In [44]:
# Probamos la función de recomendación
recomendacion("Star Wars")

Películas recomendadas para 'Star Wars':
The Empire Strikes Back - Géneros: ['Adventure', 'Action', 'Science Fiction'], Puntuación: 8.2, Popularidad: 0.03556415556483729, Reparto: ['Mark Hamill', 'Harrison Ford', 'Carrie Fisher', 'Billy Dee Williams', 'Anthony Daniels'], Director: ['Irvin Kershner']
Return of the Jedi - Géneros: ['Adventure', 'Action', 'Science Fiction'], Puntuación: 7.9, Popularidad: 0.026641824223976377, Reparto: ['Mark Hamill', 'Harrison Ford', 'Carrie Fisher', 'Billy Dee Williams', 'Anthony Daniels'], Director: ['Richard Marquand']
Star Wars: The Force Awakens - Géneros: ['Action', 'Adventure', 'Science Fiction', 'Fantasy'], Puntuación: 7.5, Popularidad: 0.05776564196080771, Reparto: ['Daisy Ridley', 'John Boyega', 'Adam Driver', 'Harrison Ford', 'Oscar Isaac'], Director: ['J.J. Abrams']
Star Wars: Episode I - The Phantom Menace - Géneros: ['Adventure', 'Action', 'Science Fiction'], Puntuación: 6.4, Popularidad: 0.028583425540174737, Reparto: ['Liam Neeson', 'Ewan 

In [45]:
import pickle

# Serializamos la matriz con pickle
pickle.dump(weighted_similarity, open("Datasets/weighted_similarity.pkl", "wb"))


## Nota aclaratoria

Se decide utilizar la mitad del dataset para obtener una matriz de entrenamiendo, con ello se genera el archivo sim.pkl con la finalidad de no realizar las iteraciones del modelo con cada vez que se busque una película y así evitar los errores de memoria en Render

In [46]:
# Calcular el número de filas para la mitad del DataFrame
half_index = len(movies_df) // 2

# Dividir el DataFrame en dos mitades
first_half = movies_df.iloc[:half_index]
second_half = movies_df.iloc[half_index:]

# Usar solo la primera mitad (o puedes trabajar con `second_half` si prefieres la otra mitad)
movies_df2 = first_half

In [47]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

# Normalizar 'vote_count' y 'popularity' para ponderación
scaler = MinMaxScaler()
movies_df2[['vote_count', 'popularity']] = scaler.fit_transform(movies_df2[['vote_count', 'popularity']])

# Crear una columna combinada para 'genres', 'overview', 'cast', y 'crew' para similitud de contenido
movies_df2['content'] = (
    movies_df2['genres'].fillna('') + " " +
    movies_df2['overview'].fillna('') + " " +
    movies_df2['cast'].fillna('') + " " +
    movies_df2['crew'].fillna('')
)

# Vectorizar el texto combinado con TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_df2['content'])

# Vectorizar únicamente 'genres' para calcular la similitud específica de géneros
tfidf_genres = TfidfVectorizer(stop_words='english')
genres_matrix = tfidf_genres.fit_transform(movies_df2['genres'].fillna(''))

# Calcular la similitud de coseno en ambas matrices (contenido y géneros)
content_similarity = cosine_similarity(tfidf_matrix)
genre_similarity = cosine_similarity(genres_matrix)

# Construir la matriz de similitud ponderada sumando los pesos de 'vote_average', 'popularity' y 'genres'
weighted_similarity2 = (
    content_similarity +
    (movies_df2['vote_average'].values[:, None] * 0.25) +
    (movies_df2['popularity'].values[:, None] * 0.5) +
    (genre_similarity * 0.25)  # Ponderación de géneros
)

# Serializamos la matriz con pickle
pickle.dump(weighted_similarity2.astype(np.float16), open("Datasets/sim.pkl", "wb"))

h:\Sistema-de-Recomendacion-de-Peliculas\venv\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
h:\Sistema-de-Recomendacion-de-Peliculas\venv\lib\site-packages\pandas\core\indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
h:\Sistema-de-Recomendacion-de-Peliculas\venv\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a c